In [1]:
#VisualPulse
#Devin Holz, Troy Cunningham, Cada Kato

import numpy as np
import cv2
import sys

#initialize webcam capture
cap = cv2.VideoCapture(0)

# Set video size to 640 by 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("Video size: ", frame_width, "x", frame_height)

#image properties
FrameRate = 30
videoChannels = 3
levels = 3
minFrequency = 1.0
maxFrequency = 2.0
bufferSize = 30
bufferIndex = 0
alpha = 50


def buildGauss(frame, levels):
    pyramid = [frame]
    for level in range(levels):
        frame = cv2.pyrDown(frame)
        pyramid.append(frame)
    return pyramid

def reconstructFrame(pyramid, index, levels):
    filteredFrame = pyramid[index]
    for level in range(levels):
        filteredFrame = cv2.pyrUp(filteredFrame)
    filteredFrame = filteredFrame[:frame_height, :frame_width]
    return filteredFrame

# Initialize Gaussian Pyramid
firstFrame = np.zeros((frame_height, frame_width, videoChannels))
firstGauss = buildGauss(firstFrame, levels+1)[levels]
videoGauss = np.zeros((bufferSize, firstGauss.shape[0], firstGauss.shape[1], videoChannels))
fourierTransformAvg = np.zeros((bufferSize))

# Bandpass Filter for Specified Frequencies
frequencies = (1.0*FrameRate) * np.arange(bufferSize) / (1.0*bufferSize)
mask = (frequencies >= minFrequency) & (frequencies <= maxFrequency)

# Heart Rate Calculation Variables
bpmCalculationFrequency = 15
bpmBufferIndex = 0
bpmBufferSize = 10
bpmBuffer = np.zeros((bpmBufferSize))


#Text Properties
font = cv2.FONT_HERSHEY_SIMPLEX
loadingTextLocation = (20, 30)
bpmTextLocation = (frame_width//2 + 5, 30)
fontScale = 1
fontColor = (255,255,255)
lineType = 2
boxColor = (0, 255, 0)
boxWeight = 3




def main(bufferIndex):
    
    while(cap.isOpened()):

        #reads frame from camera
        ret, frame = cap.read()

        if ret == True:
            
            # Construct Gaussian Pyramid
            videoGauss[bufferIndex] = buildGauss(frame, levels+1)[levels]
            fourierTransform = np.fft.fft(videoGauss, axis=0)

            # Bandpass Filter
            fourierTransform[mask == False] = 0

            # Amplify
            filtered = np.real(np.fft.ifft(fourierTransform, axis=0))
            filtered = filtered * alpha

            # Reconstruct Resulting Frame
            filteredFrame = reconstructFrame(filtered, bufferIndex, levels)
            outputFrame = frame + filteredFrame
            outputFrame = cv2.convertScaleAbs(outputFrame)

            bufferIndex = (bufferIndex + 1) % bufferSize
            

            # Display the resulting frame
            cv2.imshow('VisualPulse', outputFrame)

            # Press Q on keyboard to exit
            if cv2.waitKey(20) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()


main(bufferIndex)

Video size:  640 x 480
